In [8]:
import pandas as pd
from time import sleep
import requests

In [ ]:
# Load environment variables
load_dotenv()

headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.7',
    'Authorization': 'Bearer null',
    'Connection': 'keep-alive',
    'Referer': 'https://salute.regione.veneto.it/ivgStatistiche/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-GPC': '1',
    'User-Agent': os.getenv('USER_AGENT'),
    'dnt': '1',
    'sec-ch-ua': f'"Brave";v="{os.getenv("BROWSER_VERSION")}", "Chromium";v="{os.getenv("BROWSER_VERSION")}", "Not_A Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': f'"{os.getenv("PLATFORM")}"',
}

In [15]:
aziende_ids = list(range(101, 123)) + list(range(501, 510)) + [901, 912]

anni_search = [1,2,3,4,5,6,7,8]

# Initialize empty lists to store the data
nomi = []
valori = []
totali = []
anni = []

for anno in anni_search: 
  for aziende_id in aziende_ids:
    params = {
      'annoA': anno,
      'annoDa': anno,
      'idAzienda': aziende_id,
      'idTipoGrafico': '15',
      'isCategoriaEnable': 'false',
      'isCategoriaMax1': 'false',
    }

    response = requests.get('https://salute.regione.veneto.it/ivgStatistiche/api/grafico', params=params, headers=headers)
    data = response.json()

    if 'graficoBarre' not in data or data['graficoBarre'] is None:
        print(f"Skipping azienda {aziende_id} for year {anno} - no data found")
        continue

    for ospedale in data['graficoBarre']['list']:
      nome = ospedale['name']
      valore = ospedale['OBIETTORE']
      totale = ospedale['TOTALE']
      print(f"{anno}: Found {valore} obiettori in {nome}")

      # Append each nome and valore to their respective lists
      nomi.append(nome)
      valori.append(valore)
      anni.append(anno)
      totali.append(totale)

    sleep(0.5)

# Create the DataFrame after the loop is complete
df = pd.DataFrame({
    'nome_azienda': nomi,
    'totale': totali,
    'obiettori': valori,
    'anno_id': anni
})

Skipping azienda 111 for year 1 - no data found
1: Found 20.0 obiettori in AZIENDA OSPEDALE - UNIVERSITA' PADOVA
1: Found 13.0 obiettori in AOUI VERONA BORGO TRENTO
1: Found 8.0 obiettori in AOUI DI VERONA BORGO ROMA
Skipping azienda 111 for year 2 - no data found
2: Found 22.0 obiettori in AZIENDA OSPEDALE - UNIVERSITA' PADOVA
2: Found 8.0 obiettori in AOUI VERONA BORGO TRENTO
2: Found 10.0 obiettori in AOUI DI VERONA BORGO ROMA
Skipping azienda 111 for year 3 - no data found
3: Found 8.0 obiettori in OSPEDALE DI  BELLUNO
3: Found 4.0 obiettori in OSPEDALE DI  FELTRE
3: Found 18.0 obiettori in OSPEDALE DI  TREVISO
3: Found 6.0 obiettori in OSPEDALE DI  ODERZO
3: Found 8.0 obiettori in OSPEDALE DI  CONEGLIANO
3: Found 5.0 obiettori in OSPEDALE DI VITTORIO VENETO
3: Found 9.0 obiettori in OSPEDALE DI CASTELFRANCO
3: Found 7.0 obiettori in OSPEDALE DI MONTEBELLUNA
3: Found 16.0 obiettori in OSPEDALE DI MESTRE
3: Found 6.0 obiettori in OSPEDALE DI VENEZIA
3: Found 8.0 obiettori in OSPEDAL

In [16]:
df

,nome_azienda,totale,obiettori,anno_id
0,AZIENDA OSPEDALE - UNIVERSITA' PADOVA,25.0,20.0,1
1,AOUI VERONA BORGO TRENTO,14.0,13.0,1
2,AOUI DI VERONA BORGO ROMA,13.0,8.0,1
3,AZIENDA OSPEDALE - UNIVERSITA' PADOVA,26.0,22.0,2
4,AOUI VERONA BORGO TRENTO,10.0,8.0,2
...,...,...,...,...
192,OSPEDALE DI LEGNAGO,10.0,5.0,8
193,OSPEDALE DI SAN BONIFACIO,12.0,9.0,8
194,OSPEDALE DI VILLAFRANCA,13.0,10.0,8
195,AZIENDA OSPEDALE - UNIVERSITA' PADOVA,21.0,16.0,8


In [17]:
anni_mapping = {
    1: 2015,
    2: 2016,
    3: 2017,
    4: 2018,
    5: 2019,
    6: 2020,
    7: 2021,
    8: 2022
}

# Replace the codes with the actual years
df['anno'] = df['anno_id'].map(anni_mapping)
df['obiettori'] = df['obiettori'].astype(int)
df['totale'] = df['totale'].astype(int)
df.head(2)

,nome_azienda,totale,obiettori,anno_id,anno
0,AZIENDA OSPEDALE - UNIVERSITA' PADOVA,25,20,1,2015
1,AOUI VERONA BORGO TRENTO,14,13,1,2015


In [ ]:
df.to_csv('output/obiettori.csv', index=False, encoding='UTF-8')